## 1. Importing libraries

In [148]:
import os
import warnings

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

## 2. Reading the data

In [2]:
PROJECT_DIR = "/home/pratik/Desktop/flight_price_sagemaker"
DATA_DIR = "data"

In [3]:
def get_data(name):

    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [4]:
flights = get_data("flight_price")
flights.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset have 10,683 rows.
- The dataset consists of 10 independent features and one dependent feature.
- There are 2 features `(Route, Total_Stops)` with 1 missing value.
- There are some features with incorrect data type.

## 3. Preliminary analysis

### 3.1 Check data types

In [6]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Check for duplicates

In [8]:
flights.duplicated().sum()

220

In [9]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(['Airline', 'Date_of_Journey', 'Source', 'Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- `Date_of_Journey`, `Dep_Time`, `Arrival_Time` dtype should be datetime.
- The type of duration should be numeric type --> hours + minutes
- The type of Total_Stops should be numeric --> non-stops to 0 and so on.
- There are 220 duplicates and should be removed.

## 4. Detailed Analysis

### Airline

In [10]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [11]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of the entries have incosistent values

In [12]:
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### Date_of_Journey

In [13]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [14]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [17]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [18]:
flights.Destination

0        New Delhi
1         Banglore
2           Cochin
3         Banglore
4        New Delhi
           ...    
10678     Banglore
10679     Banglore
10680        Delhi
10681    New Delhi
10682       Cochin
Name: Destination, Length: 10683, dtype: object

In [19]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Departure time

In [47]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [48]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [52]:
pd.to_datetime(flights.Dep_Time).dt.time

/tmp/ipykernel_5354/1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival time

In [58]:
pd.to_datetime(flights.Arrival_Time).dt.time

/tmp/ipykernel_5354/2994693450.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Arrival_Time).dt.time


0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

In [64]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    # make split only on first 
    .str.split(' ', n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [70]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [85]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [87]:
flights.loc[6474]

Airline                        Air India
Date_of_Journey                6/03/2019
Source                            Mumbai
Destination                    Hyderabad
Route              BOM → GOI → PNQ → HYD
Dep_Time                           16:50
Arrival_Time                       16:55
Duration                              5m
Total_Stops                      2 stops
Additional_Info                  No info
Price                              17327
Name: 6474, dtype: object

In [113]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(' ', expand=True)
    .set_axis(['hour', 'minute'], axis=1)
    .assign(
        hour=lambda df_ : (
            df_
            .hour
            .str.replace("h", "")
            .astype("int32")
            .mul(60)
        ),
        minute=lambda df_ : (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype("int32")
        )
    )
    .sum(axis=1)
    # .dtypes
    # .isna().sum()
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [116]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(' ', expand=True)
    .set_axis(['hour', 'minute'], axis=1)
    .assign(
        hour=lambda df_ : (
            df_
            .hour
            .str.replace("h", "")
            .astype("int32")
            .mul(60)
        ),
        minute=lambda df_ : (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype("int32")
        )
    )
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=[6474]))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [117]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(' ', expand=True)
        .set_axis(['hour', 'minute'], axis=1)
        .assign(
            hour=lambda df_ : (
                df_
                .hour
                .str.replace("h", "")
                .astype("int32")
                .mul(60)
            ),
            minute=lambda df_ : (
                df_
                .minute
                .str.replace("m", "")
                .fillna("0")
                .astype("int32")
            )
        )
        .sum(axis=1)
    )

### Total Stops

In [121]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [123]:
flights.Total_Stops.isna().sum()

1

In [138]:
(
    flights
    .Total_Stops
    .str.replace("non-stop", "0")
    # ? in-front of s means --> s is optional so, it will search for stop and stops
    .str.replace(" stops?", "", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional_Info

In [142]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

- While analysis duration feature, one row is with value 5 minutes and the flight is from Mumbai to Hyderabad
- This seems incorrect entry and since it's only only row, we will drop this row

## 5. Cleaning operations

In [159]:
# According to pandas, keep apply() as your last option as it is not efficient
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        # ** --> dictionary unpacking
        .assign(**{
            col: df[col].str.strip()
            # here we saying, select dtypes which are object (O)
            for col in df.select_dtypes(include="O")
        })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_ : (
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey=lambda df_ : pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time=lambda df_ : pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_ : df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_ : (
                df_
                .total_stops
                .str.replace("non-stop", "0")
                # ? in-front of s means --> s is optional so, it will search for stop and stops
                .str.replace(" stops?", "", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_ : df_.additional_info.replace("Info", "info")
        )
        .drop(columns = "route")
    )			

In [160]:
flights_cleaned = clean_data(flights)
flights_cleaned.sample(3)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
9645,Jet Airways,2019-06-03,Mumbai,Hyderabad,07:05:00,08:30:00,85,0.0,No info,8040
7117,Multiple Carriers,2019-06-24,Delhi,Cochin,07:05:00,19:15:00,730,1.0,No info,15058
372,Indigo,2019-04-09,Banglore,Delhi,20:00:00,22:50:00,170,0.0,No info,4823


## 6. Split the data

In [161]:
flights_final = flights_cleaned.sample(1000)

In [162]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [163]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


## 7. Export the subsets

In [164]:
def export_data(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

	X.join(y).to_csv(file_path, index=False)

	return pd.read_csv(file_path).head()

In [165]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-03-09,Delhi,Cochin,00:20:00,07:40:00,440,1.0,In-flight meal not included,13977
1,Jet Airways,2019-05-24,Kolkata,Banglore,20:25:00,09:45:00,800,1.0,In-flight meal not included,12384
2,Jet Airways,2019-05-09,Delhi,Cochin,18:15:00,04:25:00,610,1.0,In-flight meal not included,12373
3,Multiple Carriers,2019-06-01,Delhi,Cochin,10:00:00,01:30:00,930,1.0,No info,7198
4,Jet Airways,2019-06-01,Delhi,Cochin,21:50:00,12:35:00,885,1.0,In-flight meal not included,10262


In [166]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-21,Mumbai,Hyderabad,07:05:00,08:30:00,85,0.0,No info,8040
1,Indigo,2019-04-27,Delhi,Cochin,05:35:00,08:50:00,195,0.0,No info,6015
2,Air Asia,2019-04-27,Delhi,Cochin,16:45:00,07:10:00,865,1.0,No info,6451
3,Goair,2019-06-12,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No info,4959
4,Jet Airways,2019-05-21,Kolkata,Banglore,21:10:00,10:05:00,775,1.0,In-flight meal not included,8586


In [167]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Vistara,2019-05-09,Banglore,Delhi,17:00:00,19:35:00,155,0.0,No info,4878
1,Indigo,2019-03-09,Chennai,Kolkata,05:15:00,07:35:00,140,0.0,No info,6482
2,Jet Airways,2019-05-18,Kolkata,Banglore,08:25:00,08:15:00,1430,1.0,In-flight meal not included,9663
3,Multiple Carriers,2019-06-27,Delhi,Cochin,17:50:00,01:30:00,460,2.0,No info,18275
4,Multiple Carriers,2019-06-03,Delhi,Cochin,07:30:00,19:15:00,705,1.0,No info,9736
